## Dependencies

In [1]:
# Confirmed needed dependencies
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import random
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim
import time

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails

# Dependency for Heat Mapper
import gmaps


# Add config.py file with the following variables and cooresponding Zillow API keys
from config import Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta
# from config import google_API_Key
from config import census_API_Key

################# ONGOING EDITS TO REQUIREMENTS.MD #################
###### IF ANY ERRORS OCCUR RELATING TO MODULES OR CONFIG.PY #######
### REFER TO requirements.md TO ENSURE YOU ARE PROPERLY SETUP ####

## File inputs/outputs

In [ ]:
# These routes have changed and must be updated before using

# masterDataCLEAN_csv = "./Data/masterDataCLEAN.csv"


# randLatLon_csv = "./Data/randomLatLon.csv" 
# addressList_csv = "./Data/addressList.csv"
# masterData_csv = "./Data/masterData.csv"
# masterDFIMPORTclean_csv = "./Data/masterDFIMPORTclean.csv"
# masterData100_csv = "./Data/masterData100.csv"
# masterData1000_csv = "./Data/masterData1000.csv"

## Helpful Functions

In [2]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 20         #### Set radius in miles

points = 40000        #### Set number of lat,lon points to generate

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [ ]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

lat_lon_df = pd.read_csv("./Data/Archived/randomLatLon.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "randLatLonMap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

## Convert Coordinates to Residential Addresses

In [ ]:
#########################
##### Yuta's Blocks #####
#########################

##### Geopy Nominatim API #####
geopy.geocoders.options.default_user_agent = "ut-group-EPIC"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

In [ ]:
# Test API - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== Test Home Response:")
response = requests.get(url, params=params_1).json()
pp.pprint(response)
print("\n" + "="*60 + "\n")

In [ ]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

In [ ]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [ ]:
##### Loop Request API for Addresses / Append to lists #####
# Make sure to import time

query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
lat = []
lon = []
neighborhood = []

counter = 1
numRequests = latlon_df["lat"].count()
rSuccess = []
rFailure = []

print(f"Processing {numRequests} Requests...")

# Nominatim API Request

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }

    time.sleep(1.1)
    response = requests.get(query_url, params=params).json()

    if response['type'] == 'house' or response['type'] == 'yes':
        lat.append(response['lat'])
        lon.append(response['lon'])
        
        try:
            postcode.append(response['address']['postcode'])
        except (KeyError, IndexError):
            postcode.append("NA")
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        
        print(f"Processed Record {counter} of {numRequests}.")
        rSuccess.append(counter)
        counter += 1
        
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        rFailure.append(counter)
        counter += 1
        
print(f"Finished Requests !!!")

In [ ]:
print("Request Results:")
print("Success #:" + str(len(rSuccess)))
print("Skipped #:" + str(len(rFailure)))

In [ ]:
# Create dataframe with addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
})

# Clean up Dataframe Columns before output (Drop incomplete zipcodes, Highway streets, and Null house # or streets)
address_df = address_df[address_df['zipcode'].str.len() == 5]
address_df = address_df[address_df['zipcode'].apply(lambda x: len(str(x)) > 3)]
address_df = address_df[address_df['street'].str.contains("Highway") == False]
address_df = address_df[address_df['house #'].str.contains("NA") == False]
address_df = address_df[address_df['street'].str.contains("NA") == False]
address_df.dtypes

In [ ]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [ ]:
# Map out CSV with gmplot

addressList_csv = "./Data/addressList.csv"

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(addressList_csv["lat"], addressList_csv["lon"], 'red', size=20, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Plot Addresses on a Map

In [ ]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It plots the addresses we have selected from the random Lat Lon points
# The resulting map is saved to a file called addressMap.html

address_df = pd.read_csv("./Data/Archived/addressList.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "blue"
pointSize = 100
mapOutputFile = "addressMap.html"
df = address_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

## Zillow API Calls using Address and Zipcode

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Read in address list to run through Zillow API 
addressDF = csvDF("../data/Archived/addressList.csv")

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

print(len(addressDF))
addressDF.head()

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Drop duplicates prior to running Zillow API call
addressDF = addressDF.drop_duplicates()
print(len(addressDF))

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

############### LOOPING FUNCTION FULLY OPERATIONAL ################
####### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY #######

# Zillow API call function using address and zipcode
def zCall(API, index, address, zipcode):
    APIkey = API[index]
    zillow_data = ZillowWrapper(APIkey)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    return result


# List containers for collected property data
zid = []
addresses = []
alats = []
alons = []
valuation = []
valChange = []
valRhigh = []
valRlow = []
sqft = []
lotsqft = []
yearBuilt = []
lastSold = []
lastPrice = []


# List of Zillow API keys to loop through due to daily API call limits
zAPIs = [Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, 
         Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, 
         Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta]
index = 0
    
for row, home in addressDF.iterrows():
    address = str(addressDF["house #"][row]) + " " + str(addressDF["street"][row])
    addresses.append(address)
    zipcode = addressDF["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")
    
    result = None
    try:
        try:
            result = zCall(zAPIs, index, address, zipcode)
            print(f"{row} Success!")
        except KeyError:  ### ERROR FOR API CALL LIMIT EXCEEDED ###
            print(f"KeyError has occurred for {address}, {zipcode} (index {row}).")
            index += 1
            print(f"Proceeding to API[{index}]")
            if index >= len(zAPIs):
                print(f"API[{index}] does not exist. Need more API keys to complete analysis.")
                break
            result = zCall(zAPIs, index, address, zipcode)

    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        valChange.append(None)
        valRhigh.append(None)
        valRlow.append(None)
        sqft.append(None)
        lotsqft.append(None)
        yearBuilt.append(None)
        lastSold.append(None)
        lastPrice.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)

    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)

    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)
        
    try:    
        change = result.zestimate_value_change
        valChange.append(change)
    except:
        print(f"No valuation change found for {address}, {zipcode} (index {row}). Appending list with null values")
        valChange.append(None)
        
    try:    
        high = result.zestimate_valuation_range_high
        valRhigh.append(high)
    except:
        print(f"No valuation range high found for {address}, {zipcode} (index {row}). Appending list with null values")
        valRhigh.append(None)

    try:    
        low = result.zestimate_valuationRange_low
        valRlow.append(low)
    except:
        print(f"No valuation range low found for {address}, {zipcode} (index {row}). Appending list with null values")
        valRlow.append(None)
        
    try:
        zsqft = result.home_size
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)
        
    try:
        lot = result.property_size
        lotsqft.append(lot)
    except:
        print(f"No lot size for {address}, {zipcode} (index {row}). Appending list with null values")
        lotsqft.append(None)
        
    try:
        year = int(result.year_built)
        yearBuilt.append(year)
    except:
        print(f"No year built for {address}, {zipcode} (index {row}). Appending list with null values")
        yearBuilt.append(None)
        
    try:
        last = result.last_sold_date
        lastSold.append(last)
    except:
        print(f"No last sold date for {address}, {zipcode} (index {row}). Appending list with null values")
        lastSold.append(None)
        
    try:
        price = result.last_sold_price
        lastPrice.append(price)
    except:
        print(f"No last sold price for {address}, {zipcode} (index {row}). Appending list with null values")
        lastPrice.append(None)


## Master Dataframe Creation

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Checking to ensure lengths of lists are identical
print(len(zid))
print(len(addresses))
print(len(alats))
print(len(alons))
print(len(valuation))
print(len(valChange))
print(len(valRhigh))
print(len(valRlow))
print(len(sqft))
print(len(lotsqft))
print(len(yearBuilt))
print(len(lastSold))
print(len(lastPrice))

# Referring back to addressList_csv generated dataframe for relevant info
addressDF.head()
print(len(addressDF))

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

masterDF = pd.DataFrame({
    "zid": zid,
    "address": addresses,
    "zipcode": addressDF["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "value_change": valChange,
    "value_range_high": valRhigh,
    "value_range_low": valRlow,
    "sqft": sqft,
#     "value_sqft": valsqft,
    "lot_sqft": lotsqft,
    "year_built": yearBuilt,
    "last_sold_date": lastSold,
    "last_sold_price": lastPrice,
})

print(len(masterDF))
masterDF.head(10)

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# masterDFclean to csv
masterDF.to_csv("../data/zillowRaw.csv", index=False, header=True)
masterDF.head(30)

## Zillow Data Cleaning

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

zillowDF = csvDF("../data/zillowRaw.csv")
print(len(zillowDF))
zillowDF.head()

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# DATA CLEANING #
# Dropping duplicates
masterDFdrops = zillowDF.drop_duplicates(subset=["zid"], keep="first")
print(len(masterDFdrops))
masterDFdrops.head()

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Cleaning out None values for "valuation"
masterDFdrops = masterDFdrops.dropna(how="any", subset=["valuation"])
print(len(masterDFdrops))
masterDFdrops.head()

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Cleaning out None values for "sqft" 
masterDFdrops = masterDFdrops.dropna(how="any", subset=["sqft"])
print(len(masterDFdrops))
masterDFdrops.head()

### Calculate Value per Sqft

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Calculate "value sqft" after None value rows removed for "valuation" and "sqft"

valsqft = []
for row, value in masterDFdrops.iterrows():
    try:
        vsqft = round((masterDFdrops["valuation"][row] / masterDFdrops["sqft"][row]), 2)
        valsqft.append(vsqft)
    except: ### THIS ERROR SHOULD NO LONGER PRINT BECAUSE NONE VALUES WERE PREVIOUSLY REMOVED
        print("Cannot perform math with NoneType")
        valsqft.append(None)

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Checking to ensure lists are appropriate lengths
print(len(masterDFdrops))
print(len(valsqft))

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Adding "value sqft" column
masterDFdrops["value_sqft"] = valsqft

# Reordering columns
masterDFdrops = masterDFdrops[['zid', 'address', 'zipcode', 'alat', 'alon', 'valuation', 'value_change', 'value_range_high',
                               'value_range_low', 'sqft', 'value_sqft', 'lot_sqft', 'year_built', 'last_sold_date', 
                               'last_sold_price']]
#                                , 'tractCode', 'countyFips', 'stateFips', 'commuteTime']]
masterDFdrops.head()

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Checking high and low values with sort
masterDFdrops.sort_values(by="value_sqft", ascending=False)

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Function for finding and dropping rows with nonsense values (e.g. valuation > $10,000,000)
def dropNonsense(dataframe, columnName, minVal, maxVal):
    dropIndices = []
    for index, row in dataframe.iterrows():
        val = dataframe[columnName][index]
        if (maxVal is not None) and (val > maxVal) or (minVal is not None) and (val < minVal):
            dropIndices.append(index)
    return dataframe.drop(index=dropIndices)

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Establish reasonable values for columns
reasonableVals = [
    ("valuation", 20000, 10000000),
    ("sqft", 500, 10000),
    ("value_sqft", 0, 1500)
]

# Loop through dataframe to drop nonsense data
masterDFdrops = masterDFdrops
for entry in reasonableVals:
    masterDFdrops = dropNonsense(masterDFdrops, entry[0], entry[1], entry[2])

# Confirm count
print(len(masterDFdrops))

In [ ]:
masterDFdrops.to_csv("../data/zillowClean.csv", index=False, header=True)


In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Save to file
DFcsv(masterDFdrops, "..data/zillowClean.csv")

### Merging Master Data with Commute Times

In [ ]:
commuteDF = csvDF("./Data/masterData.csv")
commuteDF.head()
print(len(commuteDF))
print(len(masterDFcleaning))
print("Commute")
print(commuteDF.dtypes)
print("Master")
print(masterDFcleaning.dtypes)
commuteDF.head()


In [ ]:
MASTERdf = pd.merge(masterDFcleaning, 
                    commuteDF[['tractCode', 'countyFips', 'stateFips', 'commuteTime']], 
                    how="left",
                    on='Zillow ID')

In [ ]:
print(len(MASTERdf))
MASTERdf.head()

## Plot Zillow Matched Addresses on a Map

In [ ]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It plots the Zillow addresses we have matched from our randomly selected 
# The resulting map is saved to a file called addressMap.html

masterData_df = pd.read_csv("./Data/masterDataCLEAN.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "green"
pointSize = 100
mapOutputFile = "masterDataMap.html"
df = masterData_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["alat"], df["alon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

## Crime Data

In [ ]:
crimeDF = csvDF("../data/Archived/annualCrimeData2016.csv")
crimeDF.columns
print(len(crimeDF))

In [ ]:
a = 78735.0
print(a)
a = int(a)
print(a)

In [ ]:
crimeDFdrops["zipcode"].dtype

In [ ]:
# crimeDFdrops["zipcode"].nunique()
counts = crimeDFdrops.dropna(how="any", subset=["zipcode"])
print(len(crimeDFdrops))
len(counts)

In [ ]:
crimeDFzip = counts.astype({"zipcode": int})
crimeDFzip.head()

In [ ]:
crimeDFdrops = crimeDF.dropna(how="any", subset=["GO X Coordinate"])
print(len(crimeDFdrops))

In [ ]:
crimeDFdrops = crimeDFdrops.dropna(how="any", subset=["GO Y Coordinate"])
print(len(crimeDFdrops))

In [ ]:
# Convert latitude and longitude to valid values
lats = crimeDFdrops['GO X Coordinate']
clats = []

for coord in lats:
    coord /= 100000
    clats.append(coord)
    
# Confirm length of list    
print(len(clats))

In [ ]:
# Convert latitude and longitude to valid values
lons = crimeDFdrops['GO Y Coordinate']
clons = []

for coord in lons:
    coord /= 100000
    clons.append(coord)
    
# Confirm length of list    
print(len(clons))

In [ ]:
# Drop previous coordinate columns
crimeDFdrops = crimeDFdrops.drop(['Latitude', 'Longitude'], axis=1)

print(crimeDFdrops.columns)
crimeDFdrops.head()

In [ ]:
# Append adjusted coordinate columns
crimeDFdrops["clat"] = clats
crimeDFdrops["clon"] = clons
crimeDFdrops.head()

In [ ]:
crimeDFdrops = crimeDFdrops.rename(columns = {'Highest NIBRS/UCR Offense Description': "offenseCategory"})

In [ ]:
crimeDFdrops = crimeDFdrops.rename(columns = {'GO Highest Offense Desc': "offenseDetails"})

In [ ]:
crimeDFdrops = crimeDFdrops.rename(columns = {'GO Location Zip': "zipcode"})

In [ ]:
crimeDFdrops['offenseCategory'].nunique()

In [ ]:
crimeDFsorted = crimeDFdrops.sort_values('offenseCategory')
crimeDFsorted.head()

In [ ]:
crimeCounts = crimeDFsorted["offenseCategory"].value_counts()
print(len(crimeDFsorted))

In [ ]:
crimeDFzip.to_csv("../data/crimeData.csv", index=False, header=True)

In [3]:
# Read in new crime file
crimeDF = csvDF("../data/crimeData.csv")

In [4]:
crimeDF.head()

,GO Primary Key,Council District,offenseDetails,offenseCategory,GO Report Date,GO Location,Clearance Status,Clearance Date,GO District,zipcode,GO Census Tract,clat,clon
0,201610188.0,8.0,AGG ASLT ENHANC STRANGL/SUFFOC,Agg Assault,01-Jan-16,8600 W SH 71,C,12-Jan-16,D,78735,19.08,30.67322,100.62796
1,201610643.0,9.0,THEFT,Theft,01-Jan-16,219 E 6TH ST,C,04-Jan-16,G,78701,11.00,31.14957,100.70462
2,201610892.0,4.0,AGG ROBBERY/DEADLY WEAPON,Robbery,01-Jan-16,701 W LONGSPUR BLVD,N,03-May-16,E,78753,18.23,31.29181,101.06923
3,201610893.0,9.0,THEFT,Theft,01-Jan-16,404 COLORADO ST,N,22-Jan-16,G,78701,11.00,31.13643,100.70357
4,201611148.0,1.0,DEADLY CONDUCT,Agg Assault,01-Jan-16,8002 TAPO LN,N,08-Jan-16,C,78724,22.08,31.46947,100.77985


In [5]:
# Read in crime severity csv
crimeSeverityDF = csvDF("../data/crimeDescSeverity.csv")

In [9]:
# Read in crime severity csv
crimeSeverityDF2 = csvDF("../data/crimeDataSeverity.csv")

In [10]:
print(len(crimeSeverityDF))
print(len(crimeSeverityDF2))

54
37444


In [6]:
crimeSeverityDF.head()

,Level,Severity,Offense Desc
0,H+,4,AGG ASLT ENHANC STRANGL/SUFFOC
1,H+,4,AGG ASLT STRANGLE/SUFFOCATE
2,H+,4,AGG ASLT W/MOTOR VEH FAM/DAT V
3,H+,4,AGG ASSAULT
4,H+,4,AGG ASSAULT FAM/DATE VIOLENCE


In [7]:
crimeSeverityDF = crimeSeverityDF.rename(columns = {'Offense Desc': "offenseDetails"})
crimeSeverityDF.head()

,Level,Severity,offenseDetails
0,H+,4,AGG ASLT ENHANC STRANGL/SUFFOC
1,H+,4,AGG ASLT STRANGLE/SUFFOCATE
2,H+,4,AGG ASLT W/MOTOR VEH FAM/DAT V
3,H+,4,AGG ASSAULT
4,H+,4,AGG ASSAULT FAM/DATE VIOLENCE


In [11]:
fullCrimeDF = pd.merge(crimeDF, 
                       crimeSeverityDF["Severity"], 
                       how="left", 
                       on="offenseDetails")

# MASTERdf = pd.merge(masterDFcleaning, 
#                     commuteDF[['tractCode', 'countyFips', 'stateFips', 'commuteTime']], 
#                     how="left",
#                     on='Zillow ID')

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

In [8]:
print(len(crimeDF))
print(len(crimeSeverityDF))
# print(len(fullCrimeDF))

36579
54


## School Data

In [ ]:
# Seth's section

##  Commute Time Data

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Troy's section

# This section reads the masterData csv file and uses the lat lon coordinates to get the us census tract code for that address
# It then uses the tract code to access the average commute times reported for that tract


# Load in data frame from file with lat and lon
masterData_df = csvDF("../data/zillowClean.csv")


lats = masterData_df['alat']
lons = masterData_df['alon']

# Set up arrays for new data
tractCodeList = []
countyFipsList = []
stateFipsList = []


# Use us census API to get state fips, county fips, and tract code for all addr in dataframe
for lat, lon in zip(lats,lons):
    print("getting data for " + str(lat) + str(lon))
    targetUrl = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=" + str(lon) + "&y=" + str(lat) + "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    results = requests.get(targetUrl).json()
#     print(results)
    tractCodeList.append(results["result"]["geographies"]["Census Blocks"][0]["TRACT"])
    countyFipsList.append(results["result"]["geographies"]["Census Blocks"][0]["COUNTY"])
    stateFipsList.append(results["result"]["geographies"]["Census Blocks"][0]["STATE"])
    
# Load new data into masterData
masterData_df["tractCode"] = tractCodeList
masterData_df["countyFips"] = countyFipsList
masterData_df["stateFips"] = stateFipsList
    
masterData_df.head()

In [ ]:
# Commute data incomplete due to API restrictions
# Partial data saved and Zillow data will be spliced to rerun the rest of the commute data

commutePartialDF = csvDF("../data/commutePartial.csv")
commutePartialDF.head()

In [ ]:
masterData_df.head()

In [ ]:
print("Zillow: ", len(masterData_df))
print("Commute: ", len(commutePartialDF))
commutePartialDF

In [ ]:
labels = []

for n in range(8402):
    labels.append(n)
    
print(labels)

In [ ]:
newRunDF = masterData_df.drop(labels=labels, axis=0)
newRunDF

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Troy's section

# This section reads the masterData csv file and uses the lat lon coordinates to get the us census tract code for that address
# It then uses the tract code to access the average commute times reported for that tract


# Load in data frame from file with lat and lon
# masterData_df = csvDF("../data/zillowClean.csv")


lats = newRunDF['alat']
lons = newRunDF['alon']

# Set up arrays for new data
tractCodeList2 = []
countyFipsList2 = []
stateFipsList2 = []


# Use us census API to get state fips, county fips, and tract code for all addr in dataframe
for lat, lon in zip(lats,lons):
    print("getting data for " + str(lat) + str(lon))
    targetUrl = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=" + str(lon) + "&y=" + str(lat) + "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    results = requests.get(targetUrl).json()
#     print(results)
    tractCodeList2.append(results["result"]["geographies"]["Census Blocks"][0]["TRACT"])
    countyFipsList2.append(results["result"]["geographies"]["Census Blocks"][0]["COUNTY"])
    stateFipsList2.append(results["result"]["geographies"]["Census Blocks"][0]["STATE"])
    
# Load new data into masterData
# masterData_df["tractCode"] = tractCodeList
# masterData_df["countyFips"] = countyFipsList
# masterData_df["stateFips"] = stateFipsList
    
# masterData_df.head()

In [ ]:
print(len(tractCodeList2))
print(len(countyFipsList2))
print(len(stateFipsList2))

In [ ]:
commutePart2DF = pd.DataFrame({
    "tractCode": tractCodeList2, 
    "countyFips": countyFipsList2, 
    "stateFips": stateFipsList2})
# Load new data into masterData
# commuteDF["tractCode"] = tractCodeList
# commuteDF["countyFips"] = countyFipsList
# commuteDF["stateFips"] = stateFipsList

# DFcsv(commuteDF, "../data/commutePartial.csv")

In [ ]:
fullCommuteDF = pd.concat([commutePartialDF, commutePart2DF])

In [ ]:
print(len(fullCommuteDF))

In [ ]:
tractCodeL = fullCommuteDF["tractCode"]
countyFipsL = fullCommuteDF["countyFips"]
stateFipsL = fullCommuteDF["stateFips"]

In [ ]:
tractCodeStr = tractCodeL.astype("str")
countyFipsStr = countyFipsL.astype("str")
stateFipsStr = stateFipsL.astype("str")

In [ ]:
len(tractCodeL)

In [ ]:
tractCodeListed = tractCodeStr.tolist()
countyFipsListed = countyFipsStr.tolist()
stateFipsListed = stateFipsStr.tolist()

In [ ]:
fullCommuteDF.head()
DFcsv(fullCommuteDF, "../data/commuteRaw.csv")

In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ################
####################################  SECTION NEEDING WORK ##################################################################

# Rework of commute times API call

# CSV to work with
fullCommuteDF = csvDF("../data/commuteRaw.csv")

# API key (config import malfunctioning)
census_API_Key = ""

# Container for Commute Times
commuteTimeList = []
    
for row, home in fullCommuteDF.iterrows():
    print(f"Processing index: {row}...")
    state = str(fullCommuteDF["stateFips"][row])
    county = str(fullCommuteDF["countyFips"][row])
    tract = str(fullCommuteDF["tractCode"][row])
    targetUrl = "https://api.census.gov/data/2016/acs/acs5/profile?get=DP03_0025E,NAME&for=tract:" + tract + "&in=state:" + state + " county:" + county + "&key=" + census_API_Key
    # Code errors at the following line - could be issue with targetUrl
    # No notes on where to find documentation for this API/dataset
    results = requests.get(targetUrl).json()
    print(results)
    commuteTimeList.append(results[1][0])


In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# This cell uses us census state fips, county fips, and tract code to access commute times


# Set up list to hold new data
commuteTimeList = []

loopCounter = 0

# Use us census API to get commute time from state, county and tract
for state, county, tract in zip(stateFipsListed, countyFipsListed, tractCodeListed):
    print("getting data for " + state + county + tract)
    loopCounter += 1
    print(loopCounter)
    targetUrl = "https://api.census.gov/data/2016/acs/acs5/profile?get=DP03_0025E,NAME&for=tract:" + tract + "&in=state:" + state + " county:" + county + "&key=" + census_API_Key
    results = requests.get(targetUrl).json()
    #print(results)
    commuteTimeList.append(results[1][0])

# Add commute time to masterData_df
fullCommuteDF["commuteTime"] = commuteTimeList

# write a CSV
#masterData_df.to_csv(masterData_csv, index=False, header=True)

fullCommuteDF.head()



In [ ]:
######################################################################################  RUN FOR FULL SAMPLE  ###################

# Save to file
DFcsv(masterData_df, "..data/masterData.csv")

In [ ]:
masterData_df.to_csv(masterData_csv, index=False, header=True)

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig